In [1]:
from src import visualization as vis
from src import pre_process

04-Jan-24 13:12:04 - tensorflow - WARNING  - From c:\Users\saraa\miniconda3\envs\cell\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [2]:
# vis.visualize_images("D:/TFM/Microscopy/video/3288-1.czi")

In [3]:
# vis.napari_view_splited_channels()

In [4]:
blue_df, green_df, red_df, df = pre_process.create_df("D:/TFM/Microscopy/video/copies/channels/")
df

['D:/TFM/Microscopy/video/copies/channels/blue', 'D:/TFM/Microscopy/video/copies/channels/green', 'D:/TFM/Microscopy/video/copies/channels/red']


,File_path,Image_id,Channel,Sample,Duplicates,Timepoints,Label
0,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0000,red,3288,1,0000,2
1,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0001,red,3288,1,0001,2
2,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0002,red,3288,1,0002,2
3,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0003,red,3288,1,0003,2
4,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0004,red,3288,1,0004,2
...,...,...,...,...,...,...,...
598,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0196,green,3288,1,0196,1
599,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0197,green,3288,1,0197,1
600,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0198,green,3288,1,0198,1
601,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0199,green,3288,1,0199,1


In [5]:
df.to_csv("rgb", index = True)
df

,File_path,Image_id,Channel,Sample,Duplicates,Timepoints,Label
0,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0000,red,3288,1,0000,2
1,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0001,red,3288,1,0001,2
2,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0002,red,3288,1,0002,2
3,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0003,red,3288,1,0003,2
4,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0004,red,3288,1,0004,2
...,...,...,...,...,...,...,...
598,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0196,green,3288,1,0196,1
599,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0197,green,3288,1,0197,1
600,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0198,green,3288,1,0198,1
601,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0199,green,3288,1,0199,1


In [6]:
train_x, valid_x, train_y, valid_y = pre_process.train_valid_split(df)

The proportion of train x data is: Channel
green    0.334025
red      0.334025
blue     0.331950
Name: count, dtype: float64
The proportion of valid x data is: Channel
blue     0.338843
green    0.330579
red      0.330579
Name: count, dtype: float64


In [7]:
def split_shutil_train_val():
    # 1. Creating folders 
    data_path = "D:/TFM/Microscopy/video/copies/"
    folder_names= ["train_folder", "val_folder"]
    pre_process.createfolders(data_path,folder_names)

    # 2. Spliting data
    import shutil
    dfs = [train_x, valid_x]
    paths = ["D:/TFM/Microscopy/video/copies/train_folder/", "D:/TFM/Microscopy/video/copies/val_folder/"]
    for path, df in zip(paths, dfs):
        try: 
            pre_process.move_images(df, path)
        except shutil.Error:
            print(f"The files already exist in the destination path: {path}")

split_shutil_train_val()

Folder train_folder already exists at: D:/TFM/Microscopy/video/copies/train_folder
Folder val_folder already exists at: D:/TFM/Microscopy/video/copies/val_folder


In [13]:
def create_shutil_labels():
    # 1. Creating folders 
    train_path = "D:/TFM/Microscopy/video/copies/train_folder"
    val_path = "D:/TFM/Microscopy/video/copies/val_folder"
    folder_names= ["red", "green", "blue"]
    pre_process.createfolders(train_path,folder_names)
    pre_process.createfolders(val_path,folder_names)
    
    # 2. Cretaing labels for training and validation data
    df.loc[df["Channel"] == "red", "new_file_path"] = "D:/TFM/Microscopy/video/copies/train_folder/red/" + df["Image_id"]
    df.loc[df["Channel"] == "green", "new_file_path"] = "D:/TFM/Microscopy/video/copies/train_folder/green/" + df["Image_id"] 
    df.loc[df["Channel"] == "blue", "new_file_path"] = "D:/TFM/Microscopy/video/copies/train_folder/blue/" + df["Image_id"]

    df.loc[df["Channel"] == "red", "new_file_path"] = "D:/TFM/Microscopy/video/copies/val_folder/red/" + df["Image_id"]
    df.loc[df["Channel"] == "green", "new_file_path"] = "D:/TFM/Microscopy/video/copies/val_folder/green/" + df["Image_id"] 
    df.loc[df["Channel"] == "blue", "new_file_path"] = "D:/TFM/Microscopy/video/copies/val_folder/blue/" + df["Image_id"]

    # 3. Save dataframe
    df.to_csv("rgb", index = True)


    # # 3. Cretaing labels for validating and spliting data
    # folder_path_red = "D:/TFM/Microscopy/video/copies/train_folder/red"
    # folder_path_green = "D:/TFM/Microscopy/video/copies/train_folder/green"
    # folder_path_blue = "D:/TFM/Microscopy/video/copies/train_folder/blue"
    # pre_process.images_class(df, folder_path_red, folder_path_green, folder_path_blue)    

    # folder_path_red = "D:/TFM/Microscopy/video/copies/val_folder/red"
    # folder_path_green = "D:/TFM/Microscopy/video/copies/val_folder/green"
    # folder_path_blue = "D:/TFM/Microscopy/video/copies/val_folder/blue"
    # pre_process.images_class(df, folder_path_red, folder_path_green, folder_path_blue)
    
create_shutil_labels()    

Folder red already exists at: D:/TFM/Microscopy/video/copies/train_folder\red
Folder green already exists at: D:/TFM/Microscopy/video/copies/train_folder\green
Folder blue already exists at: D:/TFM/Microscopy/video/copies/train_folder\blue
Folder red already exists at: D:/TFM/Microscopy/video/copies/val_folder\red
Folder green already exists at: D:/TFM/Microscopy/video/copies/val_folder\green
Folder blue already exists at: D:/TFM/Microscopy/video/copies/val_folder\blue
                                             File_path     Image_id Channel  \
0    D:/TFM/Microscopy/video/copies/channels/red\32...  3288-1r0000     red   
1    D:/TFM/Microscopy/video/copies/channels/red\32...  3288-1r0001     red   
2    D:/TFM/Microscopy/video/copies/channels/red\32...  3288-1r0002     red   
3    D:/TFM/Microscopy/video/copies/channels/red\32...  3288-1r0003     red   
4    D:/TFM/Microscopy/video/copies/channels/red\32...  3288-1r0004     red   
..                                                 .